In [1]:

# Import common dependencies
import pandas as pd  # noqa
import numpy as np
import matplotlib  # noqa
import matplotlib.pyplot as plt
import datetime  # noqa
import PIL  # noqa
import glob  # noqa
import pickle  # noqa
from pathlib import Path  # noqa
from scipy import misc  # noqa
import sys
import tensorflow as tf
import pdb
TRADE_COST_FRAC = .003
EPSILON = 1e-10
ADV_MULT = 1e-3

/usr/python/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
uni_tokens = set()
uni_commands = set()
uni_actions = set()
fname = 'tasks_with_length_tags.txt'
with open(fname) as f:
    content = f.readlines()
content2 = [c.split(' ') for c in content]
# you may also want to remove whitespace characters like `\n` at the end of each line
commands = []
actions = []
content = [l.replace('\n', '') for l in content]
commands = [x.split(':::')[1].split(' ')[1:-1] for x in content]
actions = [x.split(':::')[2].split(' ')[1:-2] for x in content]
structures = [x.split(':::')[3].split(' ')[2:] for x in content]

structures = [[int(l) for l in program] for program in structures]
#actions = [[wd.replace('\n', '') for wd in res] for res in actions]

In [3]:
max_actions_per_subprogram = max([max([s for s in struct]) for struct in structures]) + 1
max_actions_per_subprogram

9

In [4]:
max_num_subprograms = max([len(s) for s in structures]) + 1
max_cmd_len = max([len(s) for s in commands]) + 1
max_act_len = max([len(a) for a in actions]) + 1
max_num_subprograms, max_cmd_len, max_act_len

(7, 10, 49)

In [5]:
cmd_lengths_list = [len(s)+1 for s in commands]
cmd_lengths_np = np.array(cmd_lengths_list)

In [6]:
structures[-10], max_num_subprograms, max_actions_per_subprogram

([4, 2, 2, 2], 7, 9)

In [7]:
def build_fmap_invmap(unique, num_unique):
    fmap = dict(zip(unique, range(num_unique)))
    invmap = dict(zip(range(num_unique), unique))
    return fmap, invmap

In [8]:
for li in content2:
    for wd in li:
        uni_tokens.add(wd)

In [9]:
for li in commands:
    for wd in li:
        uni_commands.add(wd)

In [10]:
for li in actions:
    for wd in li:
        uni_actions.add(wd)

In [11]:
uni_commands.add('end_command')
uni_actions.add('end_subprogram')
uni_actions.add('end_action')

In [12]:
num_cmd = len(uni_commands)
num_act = len(uni_actions)
size_emb = 32

In [13]:
command_map, command_invmap = build_fmap_invmap(uni_commands, num_cmd)
action_map, action_invmap = build_fmap_invmap(uni_actions, num_act)

In [14]:


def dense_scaled(prev_layer, layer_size, name=None, reuse=False, scale=1.0):
    output = tf.layers.dense(prev_layer, layer_size, reuse=reuse) * scale
    return output


def dense_relu(dense_input, layer_size, scale=1.0):
    dense = dense_scaled(dense_input, layer_size, scale=scale)
    output = tf.nn.leaky_relu(dense)

    return output

def get_grad_norm(opt_fcn, loss):
    gvs = opt_fcn.compute_gradients(loss)
    grad_norm = tf.sqrt(tf.reduce_sum(
        [tf.reduce_sum(tf.square(grad)) for grad, var in gvs if grad is not None]))
    return grad_norm


def apply_clipped_optimizer(opt_fcn, loss, clip_norm=.1, clip_single=.03, clip_global_norm=False):
    gvs = opt_fcn.compute_gradients(loss)

    if clip_global_norm:
        gs, vs = zip(*[(g, v) for g, v in gvs if g is not None])
        capped_gs, grad_norm_total = tf.clip_by_global_norm([g for g in gs], clip_norm)
        capped_gvs = list(zip(capped_gs, vs))
    else:
        grad_norm_total = tf.sqrt(
            tf.reduce_sum([tf.reduce_sum(tf.square(grad)) for grad, var in gvs if grad is not None]))
        capped_gvs = [(tf.clip_by_value(grad, -1 * clip_single, clip_single), var)
                      for grad, var in gvs if grad is not None]
        capped_gvs = [(tf.clip_by_norm(grad, clip_norm), var) for grad, var in capped_gvs if grad is not None]

    optimizer = opt_fcn.apply_gradients(capped_gvs)

    return optimizer, grad_norm_total


def mlp(x, hidden_sizes, output_size=None, name='', reuse=False):
    prev_layer = x

    for idx, l in enumerate(hidden_sizes):
        dense = dense_scaled(prev_layer, l, name='mlp' + name + '_' + str(idx))
        prev_layer = tf.nn.leaky_relu(dense)

    output = prev_layer

    if output_size is not None:
        output = dense_scaled(prev_layer, output_size, name='mlp' + name + 'final')

    return output

def mlp_with_adversaries(x, hidden_sizes, output_size=None, name='', reuse=False):
    prev_layer = x
    adv_phs = []
    for idx, l in enumerate(hidden_sizes):
        
        adversary = tf.placeholder_with_default(tf.zeros_like(prev_layer), prev_layer.shape)
        prev_layer = prev_layer + adversary
        adv_phs.append(adversary)
        
        dense = dense_scaled(prev_layer, l, name='mlp' + name + '_' + str(idx))
        prev_layer = tf.nn.leaky_relu(dense)

    output = prev_layer

    if output_size is not None:
        output = dense_scaled(prev_layer, output_size, name='mlp' + name + 'final')

    return output, adv_phs



In [55]:
max_num_subprograms

7

In [56]:
tf.reset_default_graph()
cmd_mat = tf.Variable(1e-5*tf.random_normal([num_cmd, size_emb]))
act_mat = tf.Variable(1e-5*tf.random_normal([num_act, size_emb]))
global_bs = None
global_time_len = None
action_lengths = None
max_num_actions= None
# global_bs = 8
# global_time_len = 8
max_num_actions = max_actions_per_subprogram
cmd_ind = tf.placeholder(tf.int32, shape=(global_bs, global_time_len,))
act_ind = tf.placeholder(tf.int32, shape=(global_bs, global_time_len, 9))
mask_ph = tf.placeholder(tf.float32, shape=(global_bs, global_time_len, 9))
cmd_lengths = tf.placeholder(tf.int32, shape=(global_bs))
act_lengths = tf.placeholder(tf.int32, shape=(global_bs, 7))

cmd_emb = tf.nn.embedding_lookup(cmd_mat, cmd_ind)
act_emb = tf.nn.embedding_lookup(act_mat, act_ind)


num_layers_encoder = 3
hidden_filters = 16
first_cell_encoder = [tf.nn.rnn_cell.LSTMCell(
    hidden_filters, forget_bias=1., name = 'layer1_'+d) for d in ['f', 'b']]
hidden_cells_encoder = [[tf.nn.rnn_cell.LSTMCell(
    hidden_filters,forget_bias=1., name = 'layer' + str(lidx) + '_' + d)  for d in ['f', 'b']]
                        for lidx in range(num_layers_encoder - 1)]
cells_encoder = [first_cell_encoder] + hidden_cells_encoder
c1, c2 = zip(*cells_encoder)
cells_encoder = [c1, c2]
def encode(x, num_layers, cells, initial_states, lengths, name='',):
    prev_layer = x
    shortcut = x
    hiddenlayers = []
    returncells = []
    cell_fw, cell_bw = cells
    bs = tf.shape(x)[0]
    for idx in range(num_layers):
        prev_layer, c = tf.nn.bidirectional_dynamic_rnn(
                cell_fw = cell_fw[idx],
                cell_bw = cell_bw[idx],
                inputs = prev_layer,
                sequence_length=lengths,
                initial_state_fw=None,
                initial_state_bw=None,
                dtype=tf.float32,
                scope='encoder'+str(idx)
            )
        prev_layer = tf.concat(prev_layer, 2)
        prev_layer = tf.nn.leaky_relu(prev_layer)
        returncells.append(c)
        hiddenlayers.append(prev_layer)
        if idx == num_layers - 1:
            #pdb.set_trace()
            output = tf.gather_nd(
                        prev_layer,
                        tf.stack([tf.range(bs), lengths], 1),
                        name=None
                    )
            return prev_layer, returncells, hiddenlayers, output
        prev_layer = tf.concat((prev_layer, shortcut), 2)
encoding_last_layer, encoding_final_cells, encoding_hidden_layers, encoding_last_timestep = encode(
    cmd_emb, num_layers_encoder, cells_encoder,None, lengths = cmd_lengths, name = 'encoder')
# encoding_last_timestep = encoding_last_layer[:,cmd_lengths, :]
num_layers_subprogram = 3
hidden_filters_encoder = encoding_last_timestep.shape[-1].value
hidden_filters_subprogram = 16
first_cell_subprogram = tf.nn.rnn_cell.LSTMCell(
    hidden_filters_subprogram, forget_bias=1., name = 'subpogramlayer1_')
hidden_cells_subprogram = [tf.nn.rnn_cell.LSTMCell(
    hidden_filters_subprogram,forget_bias=1., name = 'subpogramlayer' + str(lidx))
                        for lidx in range(num_layers_subprogram - 1)]
cells_subprogram = [first_cell_subprogram] + hidden_cells_subprogram


def subprogram(x, num_layers, cells, initial_states, lengths, name='',):
    prev_layer = x
    shortcut = x
    hiddenlayers = []
    returncells = []
    bs = tf.shape(x)[0]
    for idx in range(num_layers):
        prev_layer, c = tf.nn.dynamic_rnn(
                cell = cells[idx],
                inputs = prev_layer,
                sequence_length=lengths,
                initial_state = None,
                dtype=tf.float32,
                scope = name + 'subprogram' + str(idx)
            )
        prev_layer = tf.concat(prev_layer, 2)
        prev_layer = tf.nn.leaky_relu(prev_layer)
        returncells.append(c)
        hiddenlayers.append(prev_layer)
        if idx == num_layers - 1:
            output = tf.gather_nd(
                        prev_layer,
                        tf.stack([tf.range(bs), lengths], 1),
                        name=None
                    )
            return prev_layer, returncells, hiddenlayers, output
        prev_layer = tf.concat((prev_layer, shortcut), 2)
encodings = [encoding_last_timestep]
last_encoding = encoding_last_timestep
initial_cmb_encoding = last_encoding
loss = 0
action_probabilities_presoftmax = []
for sub_idx in range(max_num_subprograms):      
    
    subprogram_last_layer, _, subprogram_hidden_layers, subprogram_output = subprogram(
        tf.tile(tf.expand_dims(tf.concat((last_encoding, initial_cmb_encoding), 1), 1), [1, max_num_actions, 1]), 
        num_layers_subprogram, cells_subprogram,None, lengths = act_lengths[:, sub_idx], name = 'subprogram')
    action_prob_flat = mlp(
        tf.reshape(subprogram_last_layer, [-1, hidden_filters_subprogram]),
        [32,], output_size = num_act, name = 'action_choice_mlp', reuse = (sub_idx > 0))
    action_prob_expanded = tf.reshape(action_prob_flat, [-1, max_num_actions, hidden_filters_subprogram])
    action_probabilities_layer = tf.nn.softmax(action_prob_expanded, axis=-1)
    action_probabilities_presoftmax.append(action_prob_expanded)
    delta = mlp(
        subprogram_output, [64], output_size = hidden_filters_encoder, name = 'global_transform',
        reuse = (sub_idx > 0)
    )
    last_encoding = last_encoding + delta
    encodings.append(last_encoding)
act_presoftmax = tf.stack(action_probabilities_presoftmax, 1)
#batch, subprogram, timestep, action_selection
logprobabilities = tf.nn.log_softmax(act_presoftmax, -1)
act_presoftmax_flat = tf.reshape(act_presoftmax, [-1, max_actions_per_subprogram, num_act])
mask_ph_flat = tf.reshape(mask_ph, [-1, max_actions_per_subprogram])
act_ind_flat = tf.reshape(act_ind, [-1, max_actions_per_subprogram])
ppl_loss = tf.contrib.seq2seq.sequence_loss(
    logits = act_presoftmax_flat,
    targets = act_ind_flat,
    weights = mask_ph_flat,
    average_across_timesteps=False,
    average_across_batch=False,
    softmax_loss_function=None,
    name=None
)
'''
self.dec_relu_shaped = tf.reshape(
                self.dec_relu, [self.batch_size *  (self.max_conv_len-1), 
                                self.max_sent_len, self.num_wds])[:,:-1,:]   
self.ppl_loss_masked = tf.contrib.seq2seq.sequence_loss(
    logits = self.dec_relu_shaped,
    targets = self.target_decode,
    weights = self.mask_flat_decode,
    average_across_timesteps=False,
    average_across_batch=False,
    softmax_loss_function=None,
    name=None
)
'''
opt_fcn = tf.train.AdamOptimizer(learning_rate=1e-3)
optimizer, grad_norm_total = apply_clipped_optimizer(opt_fcn, ppl_loss)
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [57]:
ppl_loss

<tf.Tensor 'sequence_loss/Reshape_3:0' shape=(?, ?) dtype=float32>

In [58]:

# cmd_ind = tf.placeholder(tf.int32, shape=(global_bs, global_time_len,))
# act_ind = tf.placeholder(tf.int32, shape=(global_bs, global_time_len, max_num_actions))
# mask_ph = tf.placeholder(tf.float32, shape=(global_bs, global_time_len, max_num_actions))
# cmd_lengths = tf.placeholder(tf.int32, shape=(global_bs))
# act_lengths = tf.placeholder(tf.int32, shape=(global_bs, max_num_actions))

In [59]:
command_map

{'after': 7,
 'and': 6,
 'around': 13,
 'end_command': 2,
 'jump': 8,
 'left': 10,
 'look': 1,
 'opposite': 5,
 'right': 9,
 'run': 12,
 'thrice': 0,
 'turn': 4,
 'twice': 11,
 'walk': 3}

In [60]:
commands_ind = [
    [command_map[c] for c in cmd] + [command_map['end_command']] + 
    [0] * (max_cmd_len - len(cmd) - 1) for cmd in commands]
actions_ind = [[action_map[a] for a in act] + [0] * (max_act_len - len(act)) for act in actions]
cmd_np = np.array(commands_ind)

In [61]:
max_cmd_len

10

In [62]:
actions_structured = []
mask_structured = []
for row in range(len(structures)):
    mask_row = []
    action_row = []
    act = actions_ind[row]
    struct = structures[row]
    start = 0
    for step in struct:
        end = start + step
        a = act[start:end]
        padding = max_actions_per_subprogram - step - 1
        action_row.append(a + [action_map['end_action']] + [0] * padding)
        start = end
    actions_structured.append(
        action_row + [[action_map['end_subprogram']] + [0] * (max_actions_per_subprogram - 1)] +
        [[0] * max_actions_per_subprogram] * (max_num_subprograms - len(struct))
    )
act_np = np.array(actions_structured)
struct_padded = [[sa + 1 for sa in s] + [1] + [0] * (max_num_subprograms - len(s) - 1) for s in structures]
struct_np = np.array(struct_padded)

mask_list = [[np.concatenate((np.ones(st), np.zeros(max_actions_per_subprogram - st)), 0) 
              for st in s] for s in struct_np]
mask_np = np.array(mask_list)

In [63]:
act_ind.shape

TensorShape([Dimension(None), Dimension(None), Dimension(9)])

In [64]:
mask_ph.shape

TensorShape([Dimension(None), Dimension(None), Dimension(9)])

In [65]:
[v.shape for v in [mask_np, struct_np, act_np, cmd_np]]

[(20910, 7, 9), (20910, 7), (20910, 8, 9), (20910, 10)]

In [66]:
cmd_np.shape

(20910, 10)

In [67]:
max_num_subprograms, max_actions_per_subprogram

(7, 9)

In [68]:
mask_np.shape

(20910, 7, 9)

In [69]:
structures[-1]

[2, 2, 2, 2, 2, 2]

In [70]:
cmd_ind.shape

TensorShape([Dimension(None), Dimension(None)])

In [71]:
struct_np.shape

(20910, 7)

In [72]:
max_actions_per_subprogram

9

In [73]:
struct_np.shape

(20910, 7)

In [75]:
cmd_ind.shape, cmd_np.shape

(TensorShape([Dimension(None), Dimension(None)]), (20910, 10))

In [76]:
act_ind.shape, act_np.shape

(TensorShape([Dimension(None), Dimension(None), Dimension(9)]), (20910, 8, 9))

In [77]:
mask_ph.shape, mask_np.shape

(TensorShape([Dimension(None), Dimension(None), Dimension(9)]), (20910, 7, 9))

In [78]:
act_lengths.shape, struct_np.shape

(TensorShape([Dimension(None), Dimension(7)]), (20910, 7))

In [79]:
cmd_lengths.shape, cmd_lengths_np.shape

(TensorShape(None), (20910,))

In [ ]:
sess.run()

In [74]:
num_samples = mask_np.shape[0]
bs = 16
for _ in range(1000):
    samples = np.random.choice(num_samples, size = bs)
    feed_dict = {
        cmd_ind : cmd_np[samples],
        act_ind : act_np[samples],
        mask_ph : mask_np[samples],
        act_lengths : np.clip(struct_np[samples], a_min = 1, a_max = None),
        cmd_lengths : cmd_lengths_np[samples],
    }
    _, loss = sess.run([optimizer, loss,], feed_dict)
    print(loss)

TypeError: Fetch argument 0 has invalid type <class 'int'>, must be a string or Tensor. (Can not convert a int into a Tensor or Operation.)

In [ ]:
np.clip(struct_np[samples], a_min = 1, a_max = None).shape

In [ ]:
struct_np[190,:]

In [ ]:
cmd_np.shape, act_np.shape, struct_np.shape

In [ ]:
structures

In [ ]:
struct

In [ ]:
[0] * (max_actions_per_subprogram - 1)

In [ ]:
action_map

In [ ]:
actions_structured[50]

In [ ]:
print(*actions_ind[2])

In [ ]:
cmd_np[:2, :], act_np[:2, :]

In [ ]:
commands_ind[:2], actions_ind[:2]

In [ ]:
commands[:2]

In [ ]:
actions[:2]

In [ ]:
structures[:2]

In [ ]:
command_map

In [ ]:
action_map

In [ ]:
act_prob

In [ ]:
subprogram_output

In [ ]:
subprogram_output

In [ ]:
subprogram_last_layer[:,cmd_lengths,:]

In [ ]:
encoding_last_layer

In [ ]:
tf.gather(
    encoding_last_layer,
    [1,2],
    axis=1
)

In [ ]:
tf.gather_nd(
    encoding_last_layer,
    np.array([[0,1,2,3,4], [1,4,3,2,5]]).T,
    name=None
)

In [ ]:
cmd_lengths

In [ ]:
def generate_command(sub_cmd, num_repeat):
    return sub_cmd * num_repeat

In [ ]:
def process_command(cmd):
    

In [ ]:
uni_commands

In [ ]:
uni_actions

In [ ]:
uni_tokens

In [ ]:
df.shape